In [1]:
# module import
import os
import sys
from datetime import datetime
import pickle

import numpy as np
import PIL.Image
import torch
import torchvision
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel, save_video, normalise_vid, get_cnn_features, img_deprocess, get_target_feature_shape
from activation_maximization import generate_preferred

In [2]:
net = torchvision.models.resnet50(pretrained=True)
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [3]:
# moment in time
weight_file = '../model/resnet50/moments_RGB_resnet50_imagenetpretrained.pth.tar'
net = torchvision.models.__dict__['resnet50'](num_classes=339)
checkpoint = torch.load(weight_file, map_location=lambda storage,
                                    loc: storage)
state_dict = {str.replace(k, 'module.', ''): v for k, v in checkpoint['state_dict'].items()}
net.load_state_dict(state_dict)
net.eval()

# image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])


# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255
bgr = False

In [4]:
# place 365
weight_file = '../model/resnet50/resnet50_places365.pth.tar'
net = torchvision.models.__dict__['resnet50'](num_classes=365)
checkpoint = torch.load(weight_file, map_location=lambda storage,
                                    loc: storage)
state_dict = {str.replace(k, 'module.', ''): v for k, v in checkpoint['state_dict'].items()}
net.load_state_dict(state_dict)
net.eval()

# image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255
bgr = False

In [5]:
# VGGface2
weight_file = '../model/resnet50/resnet50_ft_weight.pkl'
with open(weight_file, 'rb') as f:
    weights = pickle.load(f, encoding='latin1')
net = torchvision.models.__dict__['resnet50'](num_classes=8631)
own_state = net.state_dict()
for name, param in weights.items():
    if name in own_state:
        try:
            own_state[name].copy_(torch.from_numpy(param))
        except Exception:
            raise RuntimeError('While copying the parameter named {}, whose dimensions in the model are {} and whose '\
                               'dimensions in the checkpoint are {}.'.format(name, own_state[name].size(), param.size()))
    else:
        raise KeyError('unexpected key "{}" in state_dict'.format(name))

net.eval()
img_mean = np.array([93.5940,104.7624, 129.1863])
#img_mean =np.array([0,0,0])
img_std = np.array([1,1,1])
norm = 1
bgr = True

In [6]:
#save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_complexCNN_conv'
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [7]:
# initial image for the optimization
h, w = 299,299
h,w = 224, 224
initial_input = np.random.randint(0, 256, (h,w,3))

In [8]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [9]:
target_layer = "net.classifier[6]"
target_layer = "net.features.denseblock1.denselayer2.conv1"
#target_layer = "net.features.conv0"
#target_layer = "net.Conv2d_2a_3x3.conv"
target_layer = "fc"


In [10]:
#target layer setting
exec_str_list = [target_layer]
## obtain target feature shape
# transform input shape for torch avairable shape
initial_torch_input = torch.Tensor(initial_input.transpose(2,0, 1)[np.newaxis])
# obtain target layer activation 
feat_shape = get_target_feature_shape(net, initial_torch_input, exec_str_list)

In [11]:
feat_shape

(1, 8631)

In [12]:
# options
opts = {
    'img_mean': img_mean, #img_mean to preprocessing input image
    'img_std': img_std,   #img_std to preprocessing input image
    'norm': norm,         #if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
    
    'bgr': bgr,
    
    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start':1 , # learning rate
    'lr_end': 1 ,   # we can change learning rate linearly setteing these two parameters

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,   # we can change momentum linearly setteing these two parameters too

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,   # we can also change pixel decay linealy 

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during optimization
    'jitter_size': 32,
    
    'use_p_norm_reg': False, # use p_norm regularization
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    
    'input_size': (224,224,3),
    'initial_input': initial_input, # the initial image for the optimization (setting to None will use random noise as initial image)
    }



In [13]:
channel_list = [14,56, 123, 124, 215]

In [14]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # target units
    feat_size = feat_shape
    #y_index = int(feat_size[2]/2) # the unit in the center of feature map
    #x_index = int(feat_size[3]/2) # the unit in the center of feature map
    feature_mask = np.zeros(feat_size)
    #feature_mask[0,channel,y_index,x_index] = 1
    
    feature_mask[0,channel] = 1
    #feature_mask[0,:] = 1
    # weights for the target units
    feature_weight = np.zeros(feat_size, dtype=np.float32)
    #feature_weight[0,:] = -1000
    feature_weight[0,channel] = 1
    
    #
    preferred_stim = generate_preferred(net, exec_str_list, feature_mask=feature_mask, **opts)
    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
    np.save(os.path.join(save_path,save_name), preferred_stim)

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    #save_video(normalise_vid(clip_extreme_pixel(preferred_stim,pct=0.04)), save_name, save_path )
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))


channel=14

iter=1; mean(abs(feat))=0.156639;
iter=2; mean(abs(feat))=0.251772;
iter=3; mean(abs(feat))=0.326699;
iter=4; mean(abs(feat))=0.0430921;
iter=5; mean(abs(feat))=0.252212;
iter=6; mean(abs(feat))=0.496906;
iter=7; mean(abs(feat))=1.60038;
iter=8; mean(abs(feat))=1.13056;
iter=9; mean(abs(feat))=1.19612;
iter=10; mean(abs(feat))=1.50221;
iter=11; mean(abs(feat))=1.43046;
iter=12; mean(abs(feat))=2.97326;
iter=13; mean(abs(feat))=2.92705;
iter=14; mean(abs(feat))=2.60462;
iter=15; mean(abs(feat))=3.58362;
iter=16; mean(abs(feat))=3.93617;
iter=17; mean(abs(feat))=1.20593;
iter=18; mean(abs(feat))=4.19179;
iter=19; mean(abs(feat))=5.02211;
iter=20; mean(abs(feat))=5.53015;
iter=21; mean(abs(feat))=4.43932;
iter=22; mean(abs(feat))=1.48825;
iter=23; mean(abs(feat))=4.85929;
iter=24; mean(abs(feat))=4.96003;
iter=25; mean(abs(feat))=2.32361;
iter=26; mean(abs(feat))=5.70269;
iter=27; mean(abs(feat))=4.5478;
iter=28; mean(abs(feat))=6.56864;
iter=29; mean(abs(feat))=8.09113;
iter

iter=39; mean(abs(feat))=6.67449;
iter=40; mean(abs(feat))=6.35392;
iter=41; mean(abs(feat))=4.52924;
iter=42; mean(abs(feat))=4.8783;
iter=43; mean(abs(feat))=5.19441;
iter=44; mean(abs(feat))=6.98379;
iter=45; mean(abs(feat))=5.34971;
iter=46; mean(abs(feat))=6.4395;
iter=47; mean(abs(feat))=9.28586;
iter=48; mean(abs(feat))=6.11609;
iter=49; mean(abs(feat))=8.72069;
iter=50; mean(abs(feat))=5.99788;
iter=51; mean(abs(feat))=9.87242;
iter=52; mean(abs(feat))=6.97574;
iter=53; mean(abs(feat))=8.266;
iter=54; mean(abs(feat))=7.3092;
iter=55; mean(abs(feat))=6.74209;
iter=56; mean(abs(feat))=10.6765;
iter=57; mean(abs(feat))=7.977;
iter=58; mean(abs(feat))=7.52325;
iter=59; mean(abs(feat))=10.0557;
iter=60; mean(abs(feat))=9.47172;
iter=61; mean(abs(feat))=10.3799;
iter=62; mean(abs(feat))=11.8308;
iter=63; mean(abs(feat))=11.7273;
iter=64; mean(abs(feat))=9.80894;
iter=65; mean(abs(feat))=10.1663;
iter=66; mean(abs(feat))=11.7742;
iter=67; mean(abs(feat))=14.0133;
iter=68; mean(abs(fea

iter=78; mean(abs(feat))=12.6041;
iter=79; mean(abs(feat))=15.2053;
iter=80; mean(abs(feat))=13.2365;
iter=81; mean(abs(feat))=15.6814;
iter=82; mean(abs(feat))=10.2162;
iter=83; mean(abs(feat))=11.578;
iter=84; mean(abs(feat))=15.2626;
iter=85; mean(abs(feat))=14.9269;
iter=86; mean(abs(feat))=13.5255;
iter=87; mean(abs(feat))=14.4715;
iter=88; mean(abs(feat))=14.3286;
iter=89; mean(abs(feat))=11.7504;
iter=90; mean(abs(feat))=15.4633;
iter=91; mean(abs(feat))=19.4925;
iter=92; mean(abs(feat))=14.6611;
iter=93; mean(abs(feat))=15.1791;
iter=94; mean(abs(feat))=12.4418;
iter=95; mean(abs(feat))=15.2386;
iter=96; mean(abs(feat))=16.8602;
iter=97; mean(abs(feat))=14.5338;
iter=98; mean(abs(feat))=17.8088;
iter=99; mean(abs(feat))=15.8423;
iter=100; mean(abs(feat))=15.2897;
iter=101; mean(abs(feat))=15.9455;
iter=102; mean(abs(feat))=16.5436;
iter=103; mean(abs(feat))=16.1505;
iter=104; mean(abs(feat))=17.4806;
iter=105; mean(abs(feat))=19.0582;
iter=106; mean(abs(feat))=18.2974;
iter=107

iter=116; mean(abs(feat))=22.3069;
iter=117; mean(abs(feat))=24.2974;
iter=118; mean(abs(feat))=16.3743;
iter=119; mean(abs(feat))=22.5375;
iter=120; mean(abs(feat))=21.2603;
iter=121; mean(abs(feat))=23.416;
iter=122; mean(abs(feat))=27.4001;
iter=123; mean(abs(feat))=27.2673;
iter=124; mean(abs(feat))=24.0843;
iter=125; mean(abs(feat))=22.9926;
iter=126; mean(abs(feat))=26.6773;
iter=127; mean(abs(feat))=22.5561;
iter=128; mean(abs(feat))=24.9213;
iter=129; mean(abs(feat))=23.9271;
iter=130; mean(abs(feat))=23.2206;
iter=131; mean(abs(feat))=29.4054;
iter=132; mean(abs(feat))=26.1144;
iter=133; mean(abs(feat))=25.5215;
iter=134; mean(abs(feat))=29.9524;
iter=135; mean(abs(feat))=23.584;
iter=136; mean(abs(feat))=20.707;
iter=137; mean(abs(feat))=29.4902;
iter=138; mean(abs(feat))=26.946;
iter=139; mean(abs(feat))=23.6221;
iter=140; mean(abs(feat))=19.5546;
iter=141; mean(abs(feat))=24.6353;
iter=142; mean(abs(feat))=22.5013;
iter=143; mean(abs(feat))=26.5022;
iter=144; mean(abs(feat)

iter=154; mean(abs(feat))=22.6226;
iter=155; mean(abs(feat))=18.1482;
iter=156; mean(abs(feat))=23.4864;
iter=157; mean(abs(feat))=20.8365;
iter=158; mean(abs(feat))=23.9851;
iter=159; mean(abs(feat))=18.8115;
iter=160; mean(abs(feat))=22.1356;
iter=161; mean(abs(feat))=24.6211;
iter=162; mean(abs(feat))=19.568;
iter=163; mean(abs(feat))=23.1422;
iter=164; mean(abs(feat))=23.0608;
iter=165; mean(abs(feat))=24.4634;
iter=166; mean(abs(feat))=27.4218;
iter=167; mean(abs(feat))=22.6917;
iter=168; mean(abs(feat))=27.3035;
iter=169; mean(abs(feat))=27.0378;
iter=170; mean(abs(feat))=20.1096;
iter=171; mean(abs(feat))=26.3057;
iter=172; mean(abs(feat))=27.0722;
iter=173; mean(abs(feat))=24.4417;
iter=174; mean(abs(feat))=26.9486;
iter=175; mean(abs(feat))=23.3633;
iter=176; mean(abs(feat))=28.6618;
iter=177; mean(abs(feat))=28.986;
iter=178; mean(abs(feat))=26.9824;
iter=179; mean(abs(feat))=22.6665;
iter=180; mean(abs(feat))=30.01;
iter=181; mean(abs(feat))=27.7421;
iter=182; mean(abs(feat)

In [1]:
ee= get_cnn_features(net, torch.Tensor(preferred_stim.transpose(2,0,1)[np.newaxis]), exec_str_list)

NameError: name 'get_cnn_features' is not defined

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(ee[0].detach().numpy()[0])

In [88]:
net.classifier.weight.detach().numpy().min()

-0.29465234